In [32]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy as nltk_accuracy
from collections import defaultdict

In [3]:
# Descargar recursos necesarios
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/danyesid/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/danyesid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
path_data = "/mnt/c/Users/daniel.cardenas/Documents/Programacion/programming_green/BD_HIST_PRG.xlsx"
df = pd.read_excel(path_data)


In [12]:
list_tuplas = list(zip(df['Actividad'], df['Sistema']))

In [18]:
# Función para preprocesar el texto
def preprocesar(texto):
    stop_words = set(stopwords.words('spanish'))
    tokens = word_tokenize(texto.lower())
    tokens_filtrados = [word for word in tokens if word.isalnum() and word not in stop_words]
    return tokens_filtrados

# Preparar los datos para el clasificador
datos_preprocesados = [(preprocesar(texto), categoria) for (texto, categoria) in list_tuplas]

In [34]:
freq_dist_por_sistema = defaultdict(nltk.FreqDist)
for texto, sistema in datos_preprocesados:
    freq_dist_por_sistema[sistema].update(texto)

In [35]:
print(freq_dist_por_sistema)

defaultdict(<class 'nltk.probability.FreqDist'>, {'CH-BATERIA ALTA': FreqDist({'soporte': 283, 'campana': 281, 'retirar': 281, 'cable': 281, 'trifasico': 281, 'baterias': 225, 'actualizar': 223, 'alto': 223, 'carga': 212, 'voltaje': 210, ...}), 'CH-DIRECCION': FreqDist({'direccion': 348, 'splinder': 290, 'posicion': 259, 'juego': 228, '1': 176, '2': 164, 'holgura': 161, 'bomba': 161, 'axial': 126, 'caja': 123, ...}), 'CH-DIFERENCIAL': FreqDist({'aceite': 134, 'diferencial': 115, 'remanencia': 95, 'fuga': 81, 'speed': 71, 'cardan': 67, 'seguimiento': 66, 'ruido': 51, 'electromotor': 47, 'porta': 39, ...}), 'CH-FRENOS': FreqDist({'frenos': 554, 'pastillas': 462, 'prueba': 457, 'inhabilitacion': 445, 'puntos': 445, 'toma': 445, 'freno': 427, 'eje': 310, 'desgaste': 267, 'discos': 221, ...}), 'GR-VEHICULO': FreqDist({'carroceria': 12394, 'mtto': 12093, 'chasis': 4601, 'inspeccion': 2202, 'anual': 2199, 'periodica': 1848, 'revision': 1712, 'transmilenio': 1526, 'mantenimiento': 1385, 'alist

In [29]:
bat = [tupla for tupla in datos_preprocesados if tupla[1] == "CH-BATERIA ALTA"]

# Crear una lista de palabras tokenizadas
bat_words = []
for texto, _ in bat:
    if isinstance(texto, str):
        # Si texto es una cadena, tokenizarlo
        bat_words.extend(word_tokenize(texto.lower()))
    elif isinstance(texto, list):
        # Si texto ya está tokenizado, aplanar la lista
        bat_words.extend(texto)

# Asegurarse de que bat_words contiene solo cadenas
bat_words = [word for word in bat_words if isinstance(word, str)]


In [31]:
bat_words = nltk.FreqDist(bat_words)
top_bat_words = bat_words.most_common(200)
top_bat_words[:200]

[('soporte', 283),
 ('campana', 281),
 ('retirar', 281),
 ('cable', 281),
 ('trifasico', 281),
 ('baterias', 225),
 ('actualizar', 223),
 ('alto', 223),
 ('carga', 212),
 ('voltaje', 210),
 ('pruebas', 127),
 ('proceso', 79),
 ('revision', 76),
 ('descarga', 44),
 ('puerto', 43),
 ('pack', 16),
 ('sistema', 16),
 ('inteligente', 16),
 ('pin', 13),
 ('roto', 13),
 ('movil', 12),
 ('eficiencia', 11),
 ('recibe', 11),
 ('electromotor', 11),
 ('falla', 10),
 ('batería', 9),
 ('modulo', 9),
 ('ausencia', 9),
 ('tornillo', 9),
 ('energetica', 9),
 ('alta', 9),
 ('izquierdo', 8),
 ('conector', 8),
 ('amarre', 8),
 ('plástico', 8),
 ('sujeción', 8),
 ('8', 8),
 ('2', 8),
 ('fallas', 8),
 ('revisar', 8),
 ('puertos', 8),
 ('masas', 7),
 ('mal', 7),
 ('ab1', 6),
 ('sujeto', 6),
 ('corregir', 6),
 ('electrica', 6),
 ('masa', 5),
 ('seguimiento', 5),
 ('seguro', 5),
 ('2334', 5),
 ('rojo', 5),
 ('baterías', 4),
 ('baterais', 4),
 ('fuga', 4),
 ('rota', 4),
 ('cambio', 4),
 ('tornillos', 3),
 ('5',

In [16]:
# Dividir los datos en conjuntos de entrenamiento y prueba
train_size = int(len(datos_preprocesados) * 0.8)
train_set = datos_preprocesados[:train_size]
test_set = datos_preprocesados[train_size:]

# Entrenar el clasificador Naive Bayes
classifier = NaiveBayesClassifier.train(train_set)


In [17]:
# Evaluar el modelo
print("Precisión del modelo:", nltk_accuracy(classifier, test_set))

# Mostrar las características más informativas
classifier.show_most_informative_features(5)


Precisión del modelo: 0.6900462294134643
Most Informative Features
                 llantas = True           LL-LLA : CR-GEN =   7911.2 : 1.0
              conexiones = True           IT-SIR : CR-GEN =   6281.9 : 1.0
                   bomba = True           CH-REF : GR-VEH =   6141.0 : 1.0
               direccion = True           CH-DIR : GR-VEH =   3837.0 : 1.0
                   arnes = True           IT-SIR : CR-GEN =   3769.1 : 1.0
